# <b> Combining Datasets: merge and join</b>

In [83]:
import pandas as pd
import numpy as np

In [84]:
class display(object):
    """Display HTML representation of multiple objects"""

    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}</div>"""

    def __init__(self, *args):
        self.args = args  # Store all arguments (names of variables as strings)

    def _repr_html_(self):
        return '\n'.join(
            self.template.format(a, eval(a)._repr_html_())
            for a in self.args
        )

    def __repr__(self):
        return '\n\n'.join(
            a + '\n' + repr(eval(a))
            for a in self.args
        )

In [85]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'], 'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'], 'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


## <b>Categories of Joins</b>

### <b>One-to-One Joins</b>

In [86]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [87]:
pd.concat([df1, df2], axis=1)

,employee,group,employee,hire_date
0,Bob,Accounting,Lisa,2004
1,Jake,Engineering,Bob,2008
2,Lisa,Engineering,Jake,2012
3,Sue,HR,Sue,2014


In [88]:
a = df1.set_index('employee')
b = df2.set_index('employee')
print(a,'\n', b)

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR 
           hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014


In [89]:
v = df1.set_index('employee')
print(v.loc['Bob'])
print(type(v))
v.shape #(4, 1)

group    Accounting
Name: Bob, dtype: object
<class 'pandas.core.frame.DataFrame'>


(4, 1)

In [90]:
result = pd.concat(
    [df1.set_index('employee'), df2.set_index('employee')],
    axis=1
)
result

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


In [91]:
result = pd.concat(
    [df1.set_index('employee'), df2.set_index('employee')],
    axis=1
).reset_index()

'''
Explanation:
    -set_index('employee') aligns both DataFrames on the "employee" index (like how merge does it by default).
    -axis=1 stacks them side-by-side.
    -reset_index() brings "employee" back as a column.
✅ Output:
    -This will give you the same result as pd.merge(df1, df2):
'''
# result.shape #(4, 3)
result

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [92]:
s = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'], 'group': ['Accounting', 'Engineering', 'Engineering', 'HR'], 'hire_date': [2004, 2008, 2012, 2014]})
s
# s.shape  #(4, 3)

,employee,group,hire_date
0,Bob,Accounting,2004
1,Jake,Engineering,2008
2,Lisa,Engineering,2012
3,Sue,HR,2014


In [93]:
'''
By default, pd.merge() ignores the index and joins based on columns only, unless you explicitly tell it to use the index with the 'left_index' or 'right_index' parameters.
🔸 Example 1: Default merge() ignores index
'''

df01 = pd.DataFrame({
    'employee': ['Alice', 'Bob', 'Charlie'],
    'group': ['HR', 'Engineering', 'Accounting']
}, index=[1, 2, 3])

df02 = pd.DataFrame({
    'employee': ['Bob', 'Charlie', 'Alice'],
    'hire_date': [2005, 2008, 2012]
}, index=[10, 11, 12])

merged = pd.merge(df01, df02, on='employee')
merged
#▶️ You can see: the index values [1, 2, 3] and [10, 11, 12] are discarded — they're not preserved.


,employee,group,hire_date
0,Alice,HR,2012
1,Bob,Engineering,2005
2,Charlie,Accounting,2008


In [94]:
# 🔹 Example 2: Merge using index with left_index and right_index
df1_indexed = pd.DataFrame({
    'group': ['HR', 'Engineering', 'Accounting']
}, index=['Alice', 'Bob', 'Charlie'])

df2_indexed = pd.DataFrame({
    'hire_date': [2012, 2005, 2008]
}, index=['Alice', 'Bob', 'Charlie'])

merged_index = pd.merge(df1_indexed, df2_indexed,
                        left_index=True, right_index=True)
merged_index
#▶️ Now, the merge respects the index — the employee names are preserved as the index, and merging is done based on that.
#Note that you should either specify both left_index=True and right_index=True together, or neither.

,group,hire_date
Alice,HR,2012
Bob,Engineering,2005
Charlie,Accounting,2008


In [95]:
#🧪 Example 1 – one common column:
df001 = pd.DataFrame({
    'employee': ['Alice', 'Bob', 'Charlie'],
    'group': ['HR', 'Engineering', 'Accounting']
})

df002 = pd.DataFrame({
    'employee': ['Bob', 'Charlie', 'Alice'],
    'hire_date': [2005, 2008, 2012]
})

# No "on" specified
result = pd.merge(df001, df002)
result


,employee,group,hire_date
0,Alice,HR,2012
1,Bob,Engineering,2005
2,Charlie,Accounting,2008


In [96]:
#🧪 Example 2 – multiple common columns:
df_1 = pd.DataFrame({
    'name': ['A', 'B', 'C'],
    'age': [25, 30, 35],
    'score': [80, 85, 90]
})

df_2 = pd.DataFrame({
    'name': ['A', 'B', 'C'],
    'age': [25, 30, 40],  # note: C's age is different!
    'grade': ['B', 'A', 'C']
})
# 💡 Since both name and age exist in both DataFrames, it merges on both.
result = pd.merge(df_1, df_2)

#🔸 It skips 'C' because the age didn’t match.

'''
🧠 In short:
    -If you don’t use on, pandas uses all overlapping column names as keys.
    -That’s usually fine with one key, but can be risky if there are multiple shared columns unintentionally.
'''
result

,name,age,score,grade
0,A,25,80,B
1,B,30,85,A


In [97]:
pd.merge(df_1, df_2, on='name')

,name,age_x,score,age_y,grade
0,A,25,80,25,B
1,B,30,85,30,A
2,C,35,90,40,C


### <b>Many-to-One Joins</b>

In [98]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

### <b>Many-to-Many Joins</b>

In [99]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
'Engineering', 'Engineering', 'HR', 'HR'], 'skills': ['math', 'spreadsheets', 'coding', 'linux',
'spreadsheets', 'organization']})
display('df1', 'df5', 'pd.merge(df1, df5)')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


## <b>Specification of the Merge Key</b>

In [107]:
s1 = pd.Series(['a', 'b', 'c'], name='letter')
s2 = pd.Series([1, 2, 3], name='number')
print(s1, '\n', s2)

df0001 = s1.to_frame()
df0002 = s2.to_frame()
print(df0001, '\n', '\n', df0002)
merged = pd.merge(df0001, df0002, left_index=True, right_index=True)
'''
🧠 If you do specify 'left_index=True, right_index=True':

pd.merge(df1, df2, left_index=True, right_index=True)
Pandas will merge based on the index values of df1 and df2. This is useful when your index (row labels) carry meaningful data — like timestamps, IDs, or names.

❌ But if you don’t specify left_index=True, right_index=True and also don’t provide an on=... parameter:

pd.merge(df1, df2)
Then Pandas tries to merge on columns with the same name in both dataframes by default and if there are no columns with matching names in df1 and df2, then Pandas will raise a MergeError, saying:

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False.
To fix this, you need to either:

Specify a column (if there's one in common):

pd.merge(df1, df2, left_on='A', right_on='X')
Or merge on index:

pd.merge(df1, df2, left_index=True, right_index=True)
'''
print(merged)

0    a
1    b
2    c
Name: letter, dtype: object 
 0    1
1    2
2    3
Name: number, dtype: int64
  letter
0      a
1      b
2      c 
 
    number
0       1
1       2
2       3
  letter  number
0      a       1
1      b       2
2      c       3


In [112]:
dff = pd.DataFrame({'letter': s1, 'number': s2})
# dff['number'] = s2
dff

,letter,number
0,a,1
1,b,2
2,c,3


### <b>The On Key</b>

In [119]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'], 'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'], 'hire_date': [2004, 2008, 2012, 2014]})
# display('df1', 'df2', "pd.merge(df1, df2, on='employee')")
display('df1', 'df2', "pd.merge(df1, df2)")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2)
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

### <b>The left_on and right_on Keywords</b>

In [120]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue', 'Nahid'], 'group': ['Accounting', 'Engineering', 'Engineering', 'HR', 'Data Scientist']})
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue', 'Feri'],
'salary': [70000, 80000, 120000, 90000, 18000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee",right_on="name")')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
4,Nahid,Data Scientist
,name,salary
0,Bob,70000
1,Jake,80000
2,Lisa,120000
3,Sue,90000


In [124]:
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee",right_on="name").drop("name", axis = "columns")')
#or
#display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee",right_on="name").drop("name", axis = 1)')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
4,Nahid,Data Scientist
,name,salary
0,Bob,70000
1,Jake,80000
2,Lisa,120000
3,Sue,90000


### <b>The left_index and right_index Keywords</b>

In [129]:
print(df1.shape)
df1


(5, 2)


,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
4,Nahid,Data Scientist


In [132]:
# df1.set_index('employee').shape   (5, 1)
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
Nahid,Data Scientist
,hire_date
employee,
Lisa,2004
Bob,2008


In [133]:
pd.merge(df1a, df2a, left_index=True, right_index=True)

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


In [126]:
ddf1 = pd.DataFrame({'val1': [1, 2]}, index=['A', 'B'])
ddf2 = pd.DataFrame({'val2': [3, 4]}, index=['A', 'B'])
print(ddf1, '\n', ddf2)
pd.merge(ddf1, ddf2, left_index=True, right_index=True)

   val1
A     1
B     2 
    val2
A     3
B     4


,val1,val2
A,1,3
B,2,4


In [134]:
df1a.join(df2a)

,group,hire_date
employee,,
Bob,Accounting,2008.0
Jake,Engineering,2012.0
Lisa,Engineering,2004.0
Sue,HR,2014.0
Nahid,Data Scientist,NaN


In [135]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True,right_on='name')")

df1a
                   group
employee                
Bob           Accounting
Jake         Engineering
Lisa         Engineering
Sue                   HR
Nahid     Data Scientist

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000
4  Feri   18000

pd.merge(df1a, df3, left_index=True,right_on='name')
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000

## <b>Overlapping Column Names: The suffixes Keyword</b>

In [136]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
'rank': [3, 1, 4, 2]})
display('df8', 'df9', 'pd.merge(df8, df9, on="name")')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name")
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

In [137]:
display('df8', 'df9', 'pd.merge(df8, df9, on="rank")')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="rank")
  name_x  rank name_y
0    Bob     1   Jake
1   Jake     2    Sue
2   Lisa     3    Bob
3    Sue     4   Lisa

In [143]:
pd.merge(df8, df9, on='name', suffixes=['_L','_R'])

,name,rank_L,rank_R
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


## <b>Example: US States Data</b>

In [144]:
import os
#Purpose: This imports Python's os module, which provides a way to interact with the operating system. We'll use it to create directories and manage file paths.
import requests
#Purpose: This imports the requests library, which is used for making HTTP requests (like downloading files from a URL).

# Create the data directory if it doesn't exist
os.makedirs("data", exist_ok=True)
'''
Purpose: This creates a directory named 'data' if it doesn't already exist.
    -os.makedirs() is used to create directories. If the specified directory already exists, we use the exist_ok=True argument to prevent an error.
    This ensures that the data folder will be available to store the downloaded files.
'''

# Base repo URL
repo = "https://raw.githubusercontent.com/jakevdp/data-USstates/master"
'''
Purpose: This sets the base URL (repo) for the repository from where we will download the files.
It's a raw URL from GitHub that directly points to the data files in the repository.
'''

# Filenames to download
files = ["state-population.csv", "state-areas.csv", "state-abbrevs.csv"]
'''
Purpose: This creates a list of filenames to be downloaded.
These are the names of the CSV files you want to download from the repository.
'''

# Download each file
for filename in files: #Purpose: This starts a loop to iterate through each file name in the files list. The loop will allow us to download each file individually.
    url = f"{repo}/{filename}" #Purpose: This constructs the full URL for each file. f"{repo}/{filename}" creates a string by combining the base URL repo and the filename in the loop. For example, the URL for state-population.csv becomes https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv.
    response = requests.get(url) #This sends an HTTP GET request to the constructed URL and stores the response in the variable 'response'.The requests.get() method is used to retrieve the content of the file from the URL.
    with open(f"data/{filename}", "wb") as f: #This opens a file in the 'data' directory for writing in binary mode ("wb"). The f"data/{filename}" part constructs the file path for the downloaded file."wb" stands for "write binary" mode, which is necessary for saving non-text content like CSV files.
        f.write(response.content) #This writes the content of the response (which is the actual file data) to the file opened in the previous line.
    print(f"Downloaded {filename}") #This prints a message indicating that the file has been successfully downloaded. f"Downloaded {filename}" is an f-string, which dynamically inserts the filename into the string for a personalized message.


Downloaded state-population.csv
Downloaded state-areas.csv
Downloaded state-abbrevs.csv


In [145]:
pop=pd.read_csv('data/state-population.csv')
areas=pd.read_csv('data/state-areas.csv')
abbrevs=pd.read_csv('data/state-abbrevs.csv')

In [148]:
print(pop.shape)
print(areas.shape)
print(abbrevs.shape)

(2544, 4)
(52, 2)
(51, 2)


In [146]:
print(pop.head()); print(areas.head()); print(abbrevs.head())

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


In [163]:
merged = pd.merge(pop , abbrevs, how ='outer', left_on= 'state/region', right_on= 'abbreviation').drop('abbreviation', axis =1)
merged.head()
# merged.shape    #(2544, 5)

,state/region,ages,year,population,state
0,AK,total,1990,553290.0,Alaska
1,AK,under18,1990,177502.0,Alaska
2,AK,total,1992,588736.0,Alaska
3,AK,under18,1991,182180.0,Alaska
4,AK,under18,1992,184878.0,Alaska


In [161]:
AL_rows = merged[merged['state/region']== 'AL']
AL_rows.head()

,state/region,ages,year,population,state,abbreviation
48,AL,under18,2012,1117489.0,Alabama,AL
49,AL,total,2012,4817528.0,Alabama,AL
50,AL,under18,2010,1130966.0,Alabama,AL
51,AL,total,2010,4785570.0,Alabama,AL
52,AL,under18,2011,1125763.0,Alabama,AL


In [164]:
merged.isnull()

,state/region,ages,year,population,state
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
2539,False,False,False,False,False
2540,False,False,False,False,False
2541,False,False,False,False,False
2542,False,False,False,False,False


In [165]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [176]:
merged[merged['population'].isnull()]
# merged[merged['population'].isnull()].shape #(20, 5)

,state/region,ages,year,population,state
1872,PR,under18,1990,NaN,NaN
1873,PR,total,1990,NaN,NaN
1874,PR,total,1991,NaN,NaN
1875,PR,under18,1991,NaN,NaN
1876,PR,total,1993,NaN,NaN
1877,PR,under18,1993,NaN,NaN
1878,PR,under18,1992,NaN,NaN
1879,PR,total,1992,NaN,NaN
1880,PR,under18,1994,NaN,NaN
1881,PR,total,1994,NaN,NaN


In [179]:
PR_rows = merged[merged['state/region']== 'PR']
# PR_rows.shape #(48, 5)
PR_rows.head()

,state/region,ages,year,population,state
1872,PR,under18,1990,NaN,NaN
1873,PR,total,1990,NaN,NaN
1874,PR,total,1991,NaN,NaN
1875,PR,under18,1991,NaN,NaN
1876,PR,total,1993,NaN,NaN


In [195]:
merged['population'].isnull()
# .shape  (2544,)  

0       False
1       False
2       False
3       False
4       False
        ...  
2539    False
2540    False
2541    False
2542    False
2543    False
Name: population, Length: 2544, dtype: bool

In [194]:
merged['state'].isnull()
# .shape (2544,)

0       False
1       False
2       False
3       False
4       False
        ...  
2539    False
2540    False
2541    False
2542    False
2543    False
Name: state, Length: 2544, dtype: bool

In [190]:
merged.loc[ merged['state'].isnull() , 'state/region']
# .shape    (96,)

1872     PR
1873     PR
1874     PR
1875     PR
1876     PR
       ... 
2203    USA
2204    USA
2205    USA
2206    USA
2207    USA
Name: state/region, Length: 96, dtype: object

In [186]:
merged.loc[ merged['state'].isnull() , 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [181]:
merged[merged['state'].isnull()]
# merged[merged['state'].isnull()].shape #(96, 5)

,state/region,ages,year,population,state
1872,PR,under18,1990,NaN,NaN
1873,PR,total,1990,NaN,NaN
1874,PR,total,1991,NaN,NaN
1875,PR,under18,1991,NaN,NaN
1876,PR,total,1993,NaN,NaN
...,...,...,...,...,...
2203,USA,total,2010,309326295.0,NaN
2204,USA,under18,2011,73902222.0,NaN
2205,USA,total,2011,311582564.0,NaN
2206,USA,under18,2012,73708179.0,NaN


In [199]:
#These two lines are filling in missing values in the 'state' column for specific entries in 'state/region'.
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [217]:
areas

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707
5,Colorado,104100
6,Connecticut,5544
7,Delaware,1954
8,Florida,65758
9,Georgia,59441


In [209]:
areas.isnull().any()

state            False
area (sq. mi)    False
dtype: bool

In [213]:
#many-to-one join
final = pd.merge(merged, areas, on='state', how='left')
# final.shape #(2544, 6)
final.isnull().any()
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AK,total,1990,553290.0,Alaska,656425.0
1,AK,under18,1990,177502.0,Alaska,656425.0
2,AK,total,1992,588736.0,Alaska,656425.0
3,AK,under18,1991,182180.0,Alaska,656425.0
4,AK,under18,1992,184878.0,Alaska,656425.0


In [218]:
final.loc[final['area (sq. mi)'].isnull(), 'state'].unique()

array(['United States'], dtype=object)

In [224]:
# finall = final[final['population'].notna()]
# # final.shape #(2544, 6)
# # finall.shape    #(2524, 6)
# finalll = final[final['area (sq. mi)'].notna()]
# # finalll.shape   #(2496, 6)
# finallll = finall[finall['area (sq. mi)'].notna()]
# finallll.shape  #(2476, 6)

(2476, 6)

In [231]:
#✅ If you want to drop rows where both 'population' and 'area (sq. mi)' are NaN:
finall = final[final[['population', 'area (sq. mi)']].notna().any(axis=1)]
'''
Explanation:
    -final[['population', 'area (sq. mi)']] selects both columns.
    -.notna() checks for non-NaN values.
    -.any(axis=1) checks if at least one of those columns in the row is not NaN.
    -The result is a boolean mask to keep rows where at least one of the two values exists.
'''
finall.shape    #(2544, 6)

(2544, 6)

In [252]:
# If you only want to keep rows where both values are present:
final = final[final[['population', 'area (sq. mi)']].notna().all(axis=1)]
final.shape    #(2476, 6)

(2476, 7)

In [253]:
final.head()

,state/region,ages,year,population,state,area (sq. mi),density
0,AK,total,1990,553290.0,Alaska,656425.0,0.842884
1,AK,under18,1990,177502.0,Alaska,656425.0,0.270407
2,AK,total,1992,588736.0,Alaska,656425.0,0.896882
3,AK,under18,1991,182180.0,Alaska,656425.0,0.277534
4,AK,under18,1992,184878.0,Alaska,656425.0,0.281644


In [259]:
final.loc[:,'density'] = final['population'] / final['area (sq. mi)']
final.head()

,state/region,ages,year,population,state,area (sq. mi),density
0,AK,total,1990,553290.0,Alaska,656425.0,0.842884
1,AK,under18,1990,177502.0,Alaska,656425.0,0.270407
2,AK,total,1992,588736.0,Alaska,656425.0,0.896882
3,AK,under18,1991,182180.0,Alaska,656425.0,0.277534
4,AK,under18,1992,184878.0,Alaska,656425.0,0.281644


In [260]:
final = final.drop('density', axis=1)

In [281]:
#Rank US states and territories by their 2010 population density.
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
43,AK,total,2010,713868.0,Alaska,656425.0
51,AL,total,2010,4785570.0,Alabama,52423.0
141,AR,total,2010,2922280.0,Arkansas,53182.0
149,AZ,total,2010,6408790.0,Arizona,114006.0
197,CA,total,2010,37333601.0,California,163707.0


In [282]:
data2010.set_index('state', inplace = True)
# data2010.shape #(52, 5)
data2010.head()

,state/region,ages,year,population,area (sq. mi)
state,,,,,
Alaska,AK,total,2010,713868.0,656425.0
Alabama,AL,total,2010,4785570.0,52423.0
Arkansas,AR,total,2010,2922280.0,53182.0
Arizona,AZ,total,2010,6408790.0,114006.0
California,CA,total,2010,37333601.0,163707.0


In [283]:
density = data2010['population'] / data2010['area (sq. mi)']
data2010.head()

,state/region,ages,year,population,area (sq. mi)
state,,,,,
Alaska,AK,total,2010,713868.0,656425.0
Alabama,AL,total,2010,4785570.0,52423.0
Arkansas,AR,total,2010,2922280.0,53182.0
Arizona,AZ,total,2010,6408790.0,114006.0
California,CA,total,2010,37333601.0,163707.0


In [285]:
# density.shape   #(52,)
density.head()

state
Alaska          1.087509
Alabama        91.287603
Arkansas       54.948667
Arizona        56.214497
California    228.051342
dtype: float64

In [286]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [287]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64